In [1]:
# third-party
import pandas as pd
import numpy as np
import scipy
import pickle

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_fr_detection_performance, get_breath_parameters_performance
from delay_performance import normality_test, plot_inspiration_vs_expiration
from constants import CATEGORICAL_PALETTE

In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

### Activity-specific only RP analysis

In [3]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [4]:
performance_all_participants = get_breath_parameters_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,MAE Ti (s),MRE Ti (%),R^2 Ti,SSE Ti,MAE Te (s),MRE Te (%),R^2 Te,SSE Te,MAE Tb (s),MRE Tb (%),R^2 Tb,SSE Tb
0,SNB,MAG,"0.31 , 0.47","14.24 , 19.64",0.55,66.19,"0.3 , 0.43","11.16 , 14.15",0.77,51.73,"0.38 , 0.61","7.9 , 11.65",0.78,103.21
1,SNB,PZT,"0.55 , 0.58","27.41 , 24.8",0.36,123.41,"0.57 , 0.59","21.62 , 17.02",0.49,114.86,"0.45 , 0.56","9.73 , 11.04",0.75,105.46
2,SGB,MAG,"1.08 , 0.87","25.23 , 22.32",0.63,147.80,"1.14 , 0.84","20.77 , 18.37",0.55,140.76,"0.76 , 0.74","7.25 , 7.24",0.82,82.73
3,SGB,PZT,"1.39 , 1.08","32.23 , 22.87",0.10,218.41,"1.64 , 1.26","29.33 , 23.37",0.01,287.51,"0.89 , 1.11","8.7 , 11.03",0.30,155.53
4,MIXB,MAG,"0.35 , 0.44","14.33 , 16.5",0.76,40.35,"0.34 , 0.46","11.5 , 13.6",0.86,45.68,"0.34 , 0.43","6.17 , 7.13",0.94,46.66
5,MIXB,PZT,"0.77 , 0.68","30.56 , 24.48",0.64,157.77,"0.83 , 0.96","25.66 , 21.36",0.64,244.26,"0.67 , 0.83","11.67 , 12.86",0.84,170.80
6,STNB,MAG,"0.3 , 0.47","12.79 , 17.04",0.76,54.21,"0.32 , 0.48","11.87 , 16.9",0.58,59.68,"0.32 , 0.55","6.5 , 10.75",0.80,84.96
7,STNB,PZT,"0.58 , 0.53","26.7 , 21.47",0.49,108.44,"0.58 , 0.54","22.19 , 16.15",0.54,104.09,"0.39 , 0.51","8.23 , 9.59",0.80,83.30
8,MCH,MAG,"0.39 , 0.37","20.68 , 16.54",0.74,66.92,"0.36 , 0.34","19.13 , 17.53",0.71,55.18,"0.33 , 0.38","8.57 , 9.04",0.90,59.28
9,MCH,PZT,"0.64 , 0.54","31.52 , 23.47",0.46,160.05,"0.58 , 0.51","29.99 , 23.29",0.58,140.41,"0.6 , 0.58","15.26 , 14.84",0.76,157.55


In [5]:
mag = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].loc[:,["SSE Ti", "SSE Te", "SSE Tb"]].reset_index(drop=True) 
pzt = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].loc[:,["SSE Ti", "SSE Te", "SSE Tb"]].reset_index(drop=True)

a = mag / pzt 
a["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
a

,SSE Ti,SSE Te,SSE Tb,Activity
0,0.536342,0.450374,0.978665,SNB
1,0.676709,0.489583,0.531923,SGB
2,0.255752,0.187014,0.273185,MIXB
3,0.499908,0.573350,1.019928,STNB
4,0.418119,0.392992,0.376262,MCH
5,0.349215,0.286984,0.268891,SQT
6,0.409099,0.389504,0.554371,AAL
7,0.361798,0.375674,0.441570,AAR
8,0.426819,0.347194,0.311396,ALL
9,0.389116,0.483425,0.524226,ALR


### Overall RP analysis

In [6]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [7]:
get_breath_parameters_performance(overview_all, target=None)

,Sensor,MAE Ti (s),MRE Ti (%),R^2 Ti,SSE Ti,MAE Te (s),MRE Te (%),R^2 Te,SSE Te,MAE Tb (s),MRE Tb (%),R^2 Tb,SSE Tb
0,MAG,"0.39 , 0.43","19.4 , 18.79",0.74,1120.71,"0.39 , 0.43","18.2 , 17.7",0.75,1066.67,"0.35 , 0.46","8.26 , 10.13",0.90,1099.34
1,PZT,"0.62 , 0.57","31.27 , 24.39",0.54,2374.50,"0.64 , 0.61","29.56 , 23.81",0.62,2623.77,"0.55 , 0.63","13.2 , 14.43",0.83,2232.64


In [8]:
rp_detection_all = get_breath_parameters_performance(overview, target="both")
rp_detection_all

,ID,Activity,Sensor,MAE Ti (s),MRE Ti (%),R^2 Ti,SSE Ti,MAE Te (s),MRE Te (%),R^2 Te,SSE Te,MAE Tb (s),MRE Tb (%),R^2 Tb,SSE Tb
0,7OYX,SNB,MAG,"0.22 , 0.27","11.09 , 13.26",0.76,1.62,"0.23 , 0.27","10.04 , 11.73",0.71,1.63,"0.35 , 0.3","8.8 , 9.34",0.81,2.71
1,7OYX,SNB,PZT,"0.45 , 0.3","18.95 , 10.97",0.46,3.69,"0.52 , 0.33","23.35 , 15.44",0.71,3.87,"0.42 , 0.46","10.23 , 12.44",0.76,4.72
2,7OYX,SGB,MAG,"0.58 , 0.48","13.31 , 10.1",0.93,0.52,"0.76 , 0.6","12.95 , 10.75",0.84,1.12,"0.22 , 0.15","2.1 , 1.34",0.87,0.31
3,7OYX,SGB,PZT,"0.71 , 0.79","17.5 , 19.9",0.25,4.50,"1.18 , 1.61","19.89 , 28.12",0.22,15.40,"0.18 , 0.12","1.69 , 1.01",0.91,0.08
4,7OYX,MIXB,MAG,"0.1 , 0.08","5.26 , 2.66",0.98,0.10,"0.11 , 0.15","4.48 , 3.69",0.99,0.05,"0.11 , 0.06","3.11 , 1.53",0.99,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,D4GQ,UAR,PZT,"0.88 , 0.73","31.28 , 33.31",0.00,7.35,"0.93 , 0.55","34.29 , 19.8",0.07,11.58,"0.6 , 0.44","11.07 , 9.66",0.54,4.26
474,D4GQ,SE,MAG,"0.49 , 0.27","17.52 , 9.51",0.92,0.78,"0.44 , 0.32","15.19 , 10.38",0.84,0.82,"0.23 , 0.17","3.97 , 2.85",0.96,0.70
475,D4GQ,SE,PZT,"0.44 , 0.33","15.11 , 12.86",0.52,2.54,"0.81 , 0.54","30.27 , 22.11",0.03,6.07,"0.58 , 0.48","9.13 , 6.43",0.47,4.04
476,D4GQ,SS,MAG,"0.64 , 0.47","21.79 , 18.77",0.70,5.05,"0.62 , 0.35","22.77 , 11.11",0.60,4.14,"0.58 , 0.51","10.14 , 10.07",0.75,5.08


In [25]:
bp_mag = rp_detection_all[rp_detection_all["Sensor"]=="MAG"]
bp_pzt = rp_detection_all[rp_detection_all["Sensor"]=="PZT"]

,ID,Activity,Sensor,MAE Ti (s),MRE Ti (%),R^2 Ti,SSE Ti,MAE Te (s),MRE Te (%),R^2 Te,SSE Te,MAE Tb (s),MRE Tb (%),R^2 Tb,SSE Tb
0,7OYX,SNB,MAG,"0.22 , 0.27","11.09 , 13.26",0.76,1.62,"0.23 , 0.27","10.04 , 11.73",0.71,1.63,"0.35 , 0.3","8.8 , 9.34",0.81,2.71
2,7OYX,SGB,MAG,"0.58 , 0.48","13.31 , 10.1",0.93,0.52,"0.76 , 0.6","12.95 , 10.75",0.84,1.12,"0.22 , 0.15","2.1 , 1.34",0.87,0.31
4,7OYX,MIXB,MAG,"0.1 , 0.08","5.26 , 2.66",0.98,0.10,"0.11 , 0.15","4.48 , 3.69",0.99,0.05,"0.11 , 0.06","3.11 , 1.53",0.99,0.11
6,7OYX,STNB,MAG,"0.26 , 0.14","14.96 , 6.5",0.18,0.60,"0.28 , 0.15","15.44 , 7.95",0.01,1.20,"0.24 , 0.35","6.47 , 9.25",0.03,2.56
8,7OYX,MCH,MAG,"0.62 , 0.6","18.84 , 15.17",0.10,4.33,"0.63 , 0.55","22.15 , 19.71",0.31,6.41,"1.05 , 0.82","17.02 , 13.23",0.29,10.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,D4GQ,ALR,MAG,"1.3 , 0.76","44.79 , 49.12",0.92,11.25,"1.17 , 0.58","39.81 , 26.27",0.90,7.77,"1.78 , 1.32","33.46 , 31.13",0.89,26.66
470,D4GQ,UAL,MAG,"0.39 , 0.36","14.29 , 12.13",0.50,2.77,"0.53 , 0.28","20.54 , 11.76",0.54,3.59,"0.5 , 0.44","8.94 , 7.44",0.72,4.10
472,D4GQ,UAR,MAG,"0.54 , 0.46","19.26 , 16.85",0.57,4.66,"0.7 , 0.58","26.08 , 21.2",0.00,5.32,"0.75 , 0.58","12.72 , 10.26",0.66,7.40
474,D4GQ,SE,MAG,"0.49 , 0.27","17.52 , 9.51",0.92,0.78,"0.44 , 0.32","15.19 , 10.38",0.84,0.82,"0.23 , 0.17","3.97 , 2.85",0.96,0.70


In [23]:
metric = "MAE Tb (s)"
normality_test(bp_mag[metric].str.split(" , ", expand=True).astype(float).iloc[:,0], sensor=metric, type="All data")
normality_test(bp_pzt[metric].str.split(" , ", expand=True).astype(float).iloc[:,0], sensor=metric, type="All data")

In [21]:
for metric in ["MAE Tb (s)", "MAE Ti (s)", "MAE Te (s)"]:
    print(f'{metric}: {scipy.stats.wilcoxon(bp_mag[metric].str.split(" , ", expand=True).astype(float).iloc[:,0], bp_pzt[metric].str.split(" , ", expand=True).astype(float).iloc[:,0])}')

MAE Tb (s): WilcoxonResult(statistic=6519.5, pvalue=7.173929145188181e-13)
MAE Ti (s): WilcoxonResult(statistic=4554.0, pvalue=9.769842103032765e-20)
MAE Te (s): WilcoxonResult(statistic=4761.5, pvalue=9.576626182673848e-19)


In [28]:
wilc = pd.DataFrame(columns=["Activity", "Metric", "Statistic", "p-value"])

for metric in ["MAE Tb (s)", "MAE Ti (s)", "MAE Te (s)"]:
    for activity in bp_mag["Activity"].unique():
        new_entry = {}
        new_entry["Activity"] = activity
        new_entry["Metric"] = metric
        new_entry["Statistic"], new_entry["p-value"] = scipy.stats.wilcoxon(bp_mag[bp_mag["Activity"]==activity][metric].str.split(" , ", expand=True).astype(float).iloc[:,0].values, bp_pzt[bp_pzt["Activity"]==activity][metric].str.split(" , ", expand=True).astype(float).iloc[:,0].values)
        wilc.loc[len(wilc)] = new_entry
        
wilc

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching t

,Activity,Metric,Statistic,p-value
0,SNB,MAE Tb (s),50.5,0.375458
1,SGB,MAE Tb (s),59.0,0.668549
2,MIXB,MAE Tb (s),18.0,0.015076
3,STNB,MAE Tb (s),47.5,0.297852
4,MCH,MAE Tb (s),7.0,0.000580
5,SQT,MAE Tb (s),16.5,0.005157
6,AAL,MAE Tb (s),36.0,0.172848
7,AAR,MAE Tb (s),29.5,0.044312
8,ALL,MAE Tb (s),13.0,0.002686
9,ALR,MAE Tb (s),25.0,0.046826
